In [ ]:
#default_exp auth

# auth
> Helpers for creating GitHub API tokens

In [ ]:
#export
from fastcore.utils import *
from ghapi.core import *
import webbrowser,time

from urllib.parse import parse_qs

In [ ]:
#export
_def_clientid = '771f3c3af93face45f52'

## GhDeviceAuth -

In [ ]:
#export
class GhDeviceAuth(GetAttrBase):
    "Get an oauth token using the GitHub API device flow"
    _attr="params"
    def __init__(self, client_id=_def_clientid, *scopes):
        url = 'https://github.com/login/device/code'
        self.client_id = client_id
        self.params = parse_qs(urlread(url, client_id=client_id, scope=scope_str(scopes)))

    def _getattr(self,v): return v[0]

Creating a `GhDeviceAuth` will complete the first step in the GitHub API device flow, getting device and user codes.

In [ ]:
ghauth = GhDeviceAuth()
ghauth.device_code,ghauth.user_code

('29d2503c463a5e73d91fc03a6fce974b564f3b98', '879C-F118')

In [ ]:
#export
@patch
def url_docs(self:GhDeviceAuth)->str:
    "Default instructions on how to authenticate"
    return f"""First copy your one-time code: {self.user_code}
Then visit {self.verification_uri} in your browser, and paste the code when prompted."""

You can provide your own instructions on how to authenticate, or just print this out:

In [ ]:
print(ghauth.url_docs())

First copy your one-time code: 879C-F118
Then visit https://github.com/login/device in your browser, and paste the code when prompted.


In [ ]:
#export
@patch
def open_browser(self:GhDeviceAuth):
    "Open a web browser with the verification URL"
    webbrowser.open(self.verification_uri)

This uses Python's `webbrowser.open`, which will use the user's default web browser. This won't work well if the user is using a remote terminal.

In [ ]:
#export
@patch
def auth(self:GhDeviceAuth)->str:
    "Return token if authentication complete, or `None` otherwise"
    resp = parse_qs(urlread(
        'https://github.com/login/oauth/access_token',
        client_id=self.client_id, device_code=self.device_code,
        grant_type='urn:ietf:params:oauth:grant-type:device_code'))
    err = nested_idx(resp, 'error', 0)
    if err == 'authorization_pending': return None
    if err: raise Exception(resp['error_description'][0])
    return resp['access_token'][0]

Until the user has completed authentication in the browser, this will return None. Normally, you won't call this directly, but will call `wait` (see below), which will repeatedly call `auth` until authentication is complete.

In [ ]:
print(ghauth.auth())

None


In [ ]:
#export
@patch
def wait(self:GhDeviceAuth, cb:callable=None)->str:
    "Wait for authentication to complete, calling `cb` after each poll, if it is set"
    interval = int(self.interval)+1
    res = self.auth()
    while not res:
        if cb: cb()
        time.sleep(interval)
        res = self.auth()
    return res

If you pass a callback to `cb`, it will be called after each unsuccessful check for user authentication. For instance, to print a `.` to the screen after each poll, and store the token in a variable `token` when complete, you could use:

```python
token = auth.wait(lambda: print('.', end=''))
```

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_actions.ipynb.
Converted 02_auth.ipynb.
Converted 10_cli.ipynb.
Converted 50_fullapi.ipynb.
Converted 90_build_lib.ipynb.
Converted index.ipynb.
Converted tutorial_actions.ipynb.
